In [123]:
#Importing the necessary package
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import json
from datetime import datetime
import numpy as np

In [124]:
#Getting the page information
url = 'https://www.amazon.com.br/gp/bestsellers/books/'

r = requests.get(url)
soup = bs(r.content, 'html.parser')

In [125]:
#Analysing if we have all products from specific page
results = soup.find_all('div', {'class': 'a-column a-span12 a-text-center _cDEzb_grid-column_2hIsc'})
len(results)

50

### Extracting information from a product

In [126]:
# item = results[1]

# dictionary = {
#     'Position': item.find('span', {'class': 'zg-bdg-text'}).text.strip('#'),
#     'Title': item.find('div', {'class': '_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y'}).text,
#     'Author': item.find('div', {'class': 'a-row a-size-small'}).text,
#     'Stars': try:
#                 item.find('span', {'class': 'a-icon-alt'}).text.split()[0]
#              except AttributeError:
#                 return
#                 ,
#     'Reviews': item.find(lambda tag: tag.name =='span' and tag.get('class') == ['a-size-small']).text
# }   

# dictionary


In [127]:
# Creating a function to extract the informations product
def extract_information(product):
    position = product.find('span', {'class': 'zg-bdg-text'}).text.strip('#')
    title = product.find('div', {'class': '_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y'}).text
    author = product.find('div', {'class': 'a-row a-size-small'}).text
    
    try:
        stars = product.find('span', {'class': 'a-icon-alt'}).text.split()[0].replace(',','.')
    except AttributeError:
        stars=0

    try:
        reviews = product.find(lambda tag: tag.name =='span' and tag.get('class') == ['a-size-small']).text.replace('.','')
    except AttributeError:
        reviews = 0
    day = datetime.now().strftime("%Y-%m-%d %H:%M")
    

    results = [position, title, author, stars, reviews, day]

    return results

In [128]:
#Creating a dataframe with information from each product
product_infos = []

for result in results:
        product_infos.append(extract_information(result))

df = pd.DataFrame(product_infos, columns=['Position', 'Title', 'Author', 'Stars', 'Reviews', 'Date'])
df.head(5)

,Position,Title,Author,Stars,Reviews,Date
0,1,Junji Ito,Junji Ito,0,0,2022-11-13 18:31
1,2,É assim que começa (Vol. 2 É assim que acaba),Colleen Hoover,4.8,2953,2022-11-13 18:31
2,3,E não sobrou nenhum,Agatha Christie,4.7,6768,2022-11-13 18:31
3,4,É Assim que Acaba: 1,Colleen Hoover,4.8,29705,2022-11-13 18:31
4,5,Kit Hopeless,Colleen Hoover,4.8,910,2022-11-13 18:31


In [129]:
# Changing column type
df = df.astype({'Position': 'int',
           'Stars': 'float',
           'Reviews': 'int'})

df.head()

,Position,Title,Author,Stars,Reviews,Date
0,1,Junji Ito,Junji Ito,0.0,0,2022-11-13 18:31
1,2,É assim que começa (Vol. 2 É assim que acaba),Colleen Hoover,4.8,2953,2022-11-13 18:31
2,3,E não sobrou nenhum,Agatha Christie,4.7,6768,2022-11-13 18:31
3,4,É Assim que Acaba: 1,Colleen Hoover,4.8,29705,2022-11-13 18:31
4,5,Kit Hopeless,Colleen Hoover,4.8,910,2022-11-13 18:31


### Inserting results in Database

In [130]:
import pyodbc

conn = pyodbc.connect(
    "Driver={SQL Server Native Client 11.0};"
    "Server=DESKTOP-AV9L7C3;"
    "Database=Amazon;"
    "Trusted_Connection=yes;"
)

cursor = conn.cursor()

In [131]:
for i, Position in enumerate(df['Position']):
    Title = df.loc[i, 'Title']
    Author = df.loc[i, 'Author']
    Stars = df.loc[i, 'Stars']
    Reviews = df.loc[i, 'Reviews']
    Date = df.loc[i, 'Date']

    dados = str(Position) + ',\'' + Title + '\'' + ',\'' + Author + '\'' + ',' + str(Stars) + ','  + str(Reviews) + ',\'' + Date + '\')'
    script = '''INSERT INTO Product_Information (Position, Title, Author, Stars, Reviews, Extraction_Date) VALUES('''
    query = script+dados
    # print(query)
    cursor.execute(query)
    cursor.commit()
